In [67]:
import numpy as np

In [68]:
# Definindo a função sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivada da função sigmoid
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [69]:
# Inicializando os pesos e bias
input_size = 6  # Usando 3 pares de latitude e longitude (6 valores)
hidden_size = 10  # Pode ser ajustado
output_size = 10  # Prevê 5 pares de latitude e longitude (10 valores)

W1 = np.random.randn(input_size, hidden_size)
b1 = np.random.randn(hidden_size)

W2 = np.random.randn(hidden_size, output_size)
b2 = np.random.randn(output_size)

In [70]:
# Função de forward pass
def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    output = z2
    return output, a1

In [71]:
# Função de perda (Erro quadrático médio)
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [72]:
# Gerando dados fictícios de latitude e longitude
data = [
    (-0.5730, -62.6640),
    (-0.5690 , -62.6360),
    (-0.5210 , -62.6160),
    (-0.5210 , -62.6230),
    (-0.4710 , -64.4570),
]

In [73]:
X = []
y = []

for i in range(len(data) - 3):
    X.append(np.array(data[i:i+3]).flatten())  # Achata a sequência em um vetor 1D
    
    # Prever os próximos pares futuros (até onde os dados permitem)
    future_points = np.array(data[i+3:min(i+8, len(data))]).flatten()
    
    # Se não houver pares suficientes, preencher com zeros
    if len(future_points) < 10:
        future_points = np.pad(future_points, (0, 10 - len(future_points)), mode='constant')
    
    y.append(future_points)

X = np.array(X)
y = np.array(y)

In [74]:
# Verificação para garantir que os dados foram preparados corretamente
print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")

Shape de X: (2, 6)
Shape de y: (2, 10)


In [75]:
# Backpropagation e treino
learning_rate = 0.01
for epoch in range(1000):
    # Passo para frente
    y_pred, a1 = forward(X)
    
    # Cálculo do erro
    loss = mse_loss(y, y_pred)
    
    # Backpropagation
    dL_dy_pred = -2 * (y - y_pred) / len(y)
    
    # Gradientes para W2 e b2
    dL_dW2 = np.dot(a1.T, dL_dy_pred)
    dL_db2 = np.sum(dL_dy_pred, axis=0)
    
    # Gradientes para a camada oculta
    dL_da1 = np.dot(dL_dy_pred, W2.T) * sigmoid_derivative(a1)
    
    dL_dW1 = np.dot(X.T, dL_da1)
    dL_db1 = np.sum(dL_da1, axis=0)
    
    # Atualizando os pesos e biases
    W2 -= learning_rate * dL_dW2
    b2 -= learning_rate * dL_db2
    W1 -= learning_rate * dL_dW1
    b1 -= learning_rate * dL_db1
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss}")

Epoch 0, Loss: 700.4948786739981
Epoch 100, Loss: 103.95731864999998
Epoch 200, Loss: 103.95731864999998
Epoch 300, Loss: 103.95731864999998
Epoch 400, Loss: 103.95731864999998
Epoch 500, Loss: 103.95731864999998
Epoch 600, Loss: 103.95731864999998
Epoch 700, Loss: 103.95731864999998
Epoch 800, Loss: 103.95731864999998
Epoch 900, Loss: 103.95731864999998


/var/folders/34/gby6tkr103xfh0fy_017m5grcjxgrv/T/ipykernel_8365/2059585555.py:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


In [76]:
# Limites para latitude e longitude
# Aqui preciso colocar o limite do Brasil
# Evitar para que caia na água 
latitude_min, latitude_max = -22.9068, 0.80700 # altura do Brasil 
longitude_min, longitude_max = -34.79389, -73.99222 # Largura do Brasil

In [80]:
# Prever os próximos pares de coordenadas (se existirem)
if len(data) >= 3:
    new_sequence = np.array(data[-3:]).flatten()  # Usar os últimos 3 pares de coordenadas
    predicted_values, _ = forward(new_sequence)

    # Reformatar a previsão para pares de coordenadas
    predictions = predicted_values.reshape(-1, 2)

    print("Next predicted latitudes and longitudes:")
    for i, pred in enumerate(predictions):
        print(f"Prediction {i + 1}: {pred}")

Next predicted latitudes and longitudes:
Prediction 1: [ -0.496 -63.54 ]
Prediction 2: [ -0.2355 -32.2285]
Prediction 3: [0.00000000e+00 6.66133815e-16]
Prediction 4: [ 6.66133815e-16 -2.77555756e-17]
Prediction 5: [0. 0.]


/var/folders/34/gby6tkr103xfh0fy_017m5grcjxgrv/T/ipykernel_8365/2059585555.py:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
